In [2]:
!pip install pandas
!pip install selenium
!pip install scikit-learn

zsh:1: /usr/local/bin/pip: bad interpreter: /usr/local/opt/python@3.9/bin/python3.9: no such file or directory

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
zsh:1: /usr/local/bin/pip: bad interpreter: /usr/local/opt/python@3.9/bin/python3.9: no such file or directory

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
zsh:1: /usr/local/bin/pip: bad interpreter: /usr/local/opt/python@3.9/bin/python3.9: no such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 4.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 4.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def fetch_page_source(url):
    try:
        with webdriver.Chrome() as driver:
            driver.get(url)
            return driver.page_source
    except Exception as e:
        logging.error(f"Error fetching page source for URL: {url}")
        logging.exception(e)
        return None

def extract_links(page_source):
    try:
        soup = BeautifulSoup(page_source, "html.parser")
        links = soup.findAll('a', 'js__product-link-for-product-id')
        return [{'link': f"https://batdongsan.com.vn{link.get('href')}"} for link in links]
    except Exception as e:
        logging.error("Error extracting links from page source")
        logging.exception(e)
        return []

def crawl_page(url):
    logging.info(f"Crawling URL: {url}")
    page_source = fetch_page_source(url)
    if not page_source:
        logging.error(f"No page source found for URL: {url}")
        return []

    links = extract_links(page_source)
    if not links:
        logging.error(f"No links found for URL: {url}")
    return links

def crawl_links(base_url, page_range):
    all_links = []
    urls = [f"{base_url}/p{i}" for i in page_range]

    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(crawl_page, url): url for url in urls}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                links = future.result()
                all_links.extend(links)
                logging.info(f"Done processing URL: {url}")
            except Exception as e:
                logging.error(f"Error processing URL: {url}")
                logging.exception(e)

    return all_links

def save_links_to_csv(links, filename):
    df = pd.DataFrame(links)
    df.to_csv(filename, index=False)

In [ ]:
dt = crawl_links('https://batdongsan.com.vn/ban-nha-dat-ha-noi', range(1, 1000))
save_links_to_csv(dt, 'data/links/link_data.csv')

In [ ]:
page_range = range(100)

urls_and_files = [
    ('https://batdongsan.com.vn/ban-nha-rieng-ha-noi', 'data/raws/link_data.csv')
]

with ThreadPoolExecutor(max_workers=4) as executor:
    futures = {executor.submit(crawl_links, url, page_range): filename for url, filename in urls_and_files}
    for future in as_completed(futures):
        filename = futures[future]
        try:
            links = future.result()
            save_links_to_csv(links, filename)
            logging.info(f"Saved links to {filename}")
        except Exception as e:
            logging.error(f"Error saving links to {filename}")
            logging.exception(e)

In [64]:
df_link_data = pd.read_csv('data/raws/link_data.csv')
df_link_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    18000 non-null  object
dtypes: object(1)
memory usage: 140.8+ KB


In [65]:
LIST_LINK_PRODUCT = df_link_data['link'].values.tolist()
print(len(LIST_LINK_PRODUCT))

18000


In [ ]:
def extract_type_estate_and_prId(url):
    # Define regular expressions for matching prId
    prId_pattern = r"pr(\d+)"

    # Match prId using regular expression
    prId_match = re.search(prId_pattern, url)

    # Extract prId if a match is found
    prId = prId_match.group(1) if prId_match else None

    # Extract type_estate based on specific strings in the URL
    type_estate_keywords = {
        'ban-nha-rieng': 'Nhà riêng',
        'ban-nha-mat-pho': 'Nhà mặt phố',
        'ban-nha-biet-thu-lien-ke': 'Nhà biệt thự liền kề',
        'ban-shophouse-nha-pho-thuong-mai': 'Shophouse/Nhà phố thương mại'
    }

    type_estate = 'Unknown'
    for keyword, type_name in type_estate_keywords.items():
        if keyword in url:
            type_estate = type_name
            break

    return type_estate, prId

In [67]:
df = pd.read_csv('data/raws/raw_data.csv')
df.info()

critical_fields = ['district', 'posted_date', 'area', 'price']
df.dropna(subset=critical_fields, how='all', inplace=True)
df.to_csv('data/raws/raw_data.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18092 entries, 0 to 18091
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   pr_id                18092 non-null  int64 
 1   type_estate          18092 non-null  object
 2   district             18058 non-null  object
 3   posted_date          18049 non-null  object
 4   area                 18049 non-null  object
 5   price                18049 non-null  object
 6   legal_document       15696 non-null  object
 7   interior             10806 non-null  object
 8   num_bedrooms         12577 non-null  object
 9   num_bathrooms        11188 non-null  object
 10  num_floors           15969 non-null  object
 11  house_orientation    3500 non-null   object
 12  balcony_orientation  2182 non-null   object
 13  entrance             8577 non-null   object
 14  frontage             12233 non-null  object
dtypes: int64(1), object(14)
memory usage: 2.1+ MB


In [68]:
df = pd.read_csv('data/raws/raw_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18058 entries, 0 to 18057
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   pr_id                18058 non-null  int64 
 1   type_estate          18058 non-null  object
 2   district             18058 non-null  object
 3   posted_date          18049 non-null  object
 4   area                 18049 non-null  object
 5   price                18049 non-null  object
 6   legal_document       15696 non-null  object
 7   interior             10806 non-null  object
 8   num_bedrooms         12577 non-null  object
 9   num_bathrooms        11188 non-null  object
 10  num_floors           15969 non-null  object
 11  house_orientation    3500 non-null   object
 12  balcony_orientation  2182 non-null   object
 13  entrance             8577 non-null   object
 14  frontage             12233 non-null  object
dtypes: int64(1), object(14)
memory usage: 2.1+ MB


In [ ]:
LIST_PRODUCT = []

# Define a function to extract property details
def extract_property_details(url):
    property_details = {
        'pr_id': '',
        'type_estate': '',
        'district': '',
        'posted_date': '',
        'area': '',
        'price': '',
        'legal_document': '',
        'interior': '',
        'num_bedrooms': '',
        'num_bathrooms': '',
        'num_floors': '',
        'house_orientation': '',
        'balcony_orientation': '',
        'entrance': '',
        'frontage': ''
    }

    property_details['type_estate'], property_details['pr_id'] = extract_type_estate_and_prId(url)

    try:
        # Set up Selenium WebDriver
        with webdriver.Chrome() as driver:
            driver.get(url)
            
            # Get the page source
            page_source = driver.page_source

        soup = BeautifulSoup(page_source, "html.parser")

        try:
            # Find the breadcrumb element
            breadcrumb = soup.find('div', class_='re__breadcrumb')
            if breadcrumb:
                level_3_link = breadcrumb.find('a', {'level': '3'})
                if level_3_link:
                    property_details['district'] = level_3_link.text
            else:
                print(f"Breadcrumb not found for URL: {url}")
        except AttributeError as e:
            print(f"Error extracting district: {e}")

        # Extract posted date
        try:
            posted_date_span = soup.find('span', string='Ngày đăng')
            if posted_date_span:
                property_details['posted_date'] = posted_date_span.find_next_sibling('span').text.strip()
        except AttributeError as e:
            print(f"Error extracting posted_date: {e}")

        # Extract property specifications
        try:
            specs_content = soup.find_all('div', class_='re__pr-specs-content-item')
            # Mapping of titles to corresponding keys in property_details
            title_map = {
                'Diện tích': 'area',
                'Mức giá': 'price',
                'Pháp lý': 'legal_document',
                'Nội thất': 'interior',
                'Số phòng ngủ': 'num_bedrooms',
                'Số toilet': 'num_bathrooms',
                'Số tầng': 'num_floors',
                'Hướng nhà': 'house_orientation',
                'Hướng ban công': 'balcony_orientation',
                'Đường vào': 'entrance',
                'Mặt tiền': 'frontage'
            }

            # Extract and map property details
            for item in specs_content:
                title = item.find(class_='re__pr-specs-content-item-title').text.strip()
                if title in title_map:
                    property_details[title_map[title]] = item.find(class_='re__pr-specs-content-item-value').text.strip()
        except AttributeError as e:
            print(f"Error extracting property details: {e}")
        
    except Exception as e:
        print(f"Failed to extract data from {url}: {e}")

    return property_details

# Function to write property details to CSV
def write_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, mode='a', header=not pd.io.common.file_exists(filename))

# Function to get existing pr_id from CSV
def get_existing_pr_ids(filename):
    if pd.io.common.file_exists(filename):
        df = pd.read_csv(filename)
        print(df)
        existing_pr_ids = df['pr_id'].astype(str).tolist()
        return existing_pr_ids
    return []

# Function to filter URLs based on existing pr_id
def filter_urls(urls, existing_pr_ids):
    filtered_urls = []
    for url in urls:
        try:
            pr_id = extract_type_estate_and_prId(url)[1]
            if pr_id not in existing_pr_ids:
                filtered_urls.append(url)
        except Exception as e:
            print(f"Error processing URL {url}: {e}")
    return filtered_urls

def scrape_property_details(urls, filename):
    existing_pr_ids = get_existing_pr_ids(filename)
    print('existing_pr_ids', existing_pr_ids)
    # Filter URLs that are not in existing_pr_ids
    filtered_urls = filter_urls(urls, existing_pr_ids)
    print('filtered_urls', filtered_urls)
    
    scraped_properties = []
    with ThreadPoolExecutor(max_workers=7) as executor:
        futures = {executor.submit(extract_property_details, url): url for url in filtered_urls}
        try:
            for future in as_completed(futures):
                url = futures[future]
                try:
                    result = future.result()
                    scraped_properties.append(result)
                    write_to_csv([result], filename)
                except Exception as e:
                    print(f"Failed to extract data from {url}: {e}")
        except KeyboardInterrupt:
            for f in futures:
                f.cancel()
            raise

    return scraped_properties

LIST_PRODUCT = scrape_property_details(LIST_LINK_PRODUCT, 'data/raws/raw_data.csv')
print(LIST_PRODUCT)

In [69]:
df = pd.read_csv('data/raws/raw_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   pr_id                18000 non-null  int64 
 1   type_estate          18000 non-null  object
 2   district             18000 non-null  object
 3   posted_date          17991 non-null  object
 4   area                 17991 non-null  object
 5   price                17991 non-null  object
 6   legal_document       15645 non-null  object
 7   interior             10765 non-null  object
 8   num_bedrooms         12538 non-null  object
 9   num_bathrooms        11150 non-null  object
 10  num_floors           15919 non-null  object
 11  house_orientation    3491 non-null   object
 12  balcony_orientation  2174 non-null   object
 13  entrance             8551 non-null   object
 14  frontage             12201 non-null  object
dtypes: int64(1), object(14)
memory usage: 2.1+ MB


In [86]:
df = pd.read_csv('data/raws/raw_data.csv')
df.drop(['house_orientation', 'balcony_orientation'], axis=1, inplace=True)
# Remove duplicate rows based on 'pr_id'
df = df.drop_duplicates(subset='pr_id')

# Remove rows where 'price' is 'Thỏa thuận'
df = df[df['price'].str.lower() != 'thỏa thuận']

df.to_csv('data/raws/raw_data.csv', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14610 entries, 0 to 16272
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pr_id           14610 non-null  int64 
 1   type_estate     14610 non-null  object
 2   district        14610 non-null  object
 3   posted_date     14601 non-null  object
 4   area            14601 non-null  object
 5   price           14601 non-null  object
 6   legal_document  12738 non-null  object
 7   interior        8976 non-null   object
 8   num_bedrooms    10537 non-null  object
 9   num_bathrooms   9371 non-null   object
 10  num_floors      13194 non-null  object
 11  entrance        6962 non-null   object
 12  frontage        10034 non-null  object
dtypes: int64(1), object(12)
memory usage: 1.6+ MB


In [87]:
# Read the CSV data into a pandas DataFrame
df = pd.read_csv('data/raws/raw_data.csv')

# Function to get unique values and their count for specified columns
def unique_values(df, columns):
    for column in columns:
        unique_vals = df[column].unique()
        print(f"Column '{column}' has {len(unique_vals)} unique values: {unique_vals}")

# Columns to check
columns_to_check = ['type_estate', 'district', 'legal_document', 'interior']

# Get unique values and their count for the specified columns
unique_values(df, columns_to_check)

Column 'type_estate' has 4 unique values: ['Nhà riêng' 'Nhà mặt phố' 'Nhà biệt thự liền kề'
 'Shophouse/Nhà phố thương mại']
Column 'district' has 30 unique values: ['Cầu Giấy' 'Tây Hồ' 'Ba Đình' 'Hoàng Mai' 'Bắc Từ Liêm' 'Nam Từ Liêm'
 'Hà Đông' 'Thanh Xuân' 'Đống Đa' 'Hoài Đức' 'Long Biên' 'Hai Bà Trưng'
 'Quốc Oai' 'Chương Mỹ' 'Thanh Trì' 'Hoàn Kiếm' 'Thường Tín' 'Đông Anh'
 'Sơn Trà' 'Sóc Sơn' 'Gia Lâm' 'Đan Phượng' 'Mê Linh' 'Thanh Oai'
 'Thạch Thất' 'Gò Vấp' 'Phúc Thọ' 'Sơn Tây' 'Ba Vì' 'Mỹ Đức']
Column 'legal_document' has 383 unique values: ['Sổ đỏ/ Sổ hồng' 'Sổ đỏ chính chủ.' nan 'Sổ đỏ/ Sổ hồng.'
 'Hợp đồng mua bán' 'Sổ đỏ' 'Có sổ.' 'Full sổ đỏ.' 'Sổ đỏ.'
 'Sổ đỏ chính chủ' 'Sổ đỏ vuông đẹp nở hậu chút.'
 '- Sổ đỏ chính chủ, giao dịch nhanh chóng.' '- Sổ đỏ'
 '- Sổ đỏ vuông vắn nở hậu.' 'Sổ đỏ vuông vắn' 'Sổ đỏ vuông vắn.'
 'Đang chờ sổ' '+ Sổ cất két, sẵn sàng giao dịch.' '- Sổ đẹp vuông.'
 '+ Sổ đẹp sẵn sàng chờ chủ mới.' 'Sổ đỏ CC' '+ Sổ vuông,'
 '+ Sổ chính chủ, sẵn sàng 

In [7]:
# Function to standardize a column using mapping
def standardize_column(column, mapping):
    if pd.isna(column):
        return 'Khác'
    
    column_lower = column.lower()
    for key in mapping:
        if key in column_lower:
            return mapping[key]
    return 'Khác'

# Dictionary mapping for 'legal_document'
legal_document_mapping = {
    'sổ đỏ': 'Sổ đỏ/ sổ hồng',
    'sổ hồng': 'Sổ đỏ/ sổ hồng',
    'sđcc': 'Sổ đỏ/ sổ hồng',
    'sdcc': 'Sổ đỏ/ sổ hồng',
    'hợp đồng mua bán': 'Hợp đồng mua bán',
    'hđmb': 'Hợp đồng mua bán',
    'đang chờ sổ': 'Đang chờ sổ',
    'chưa có sổ': 'Đang chờ sổ',
    'chưa sổ': 'Đang chờ sổ',
}

# Dictionary mapping for 'interior'
interior_mapping = {
    'không nội thất': 'Không nội thất',
    'cơ bản': 'Cơ bản',
    'đầy đủ': 'Đầy đủ',
    'đủ đồ': 'Đầy đủ',
    'full': 'Đầy đủ',
    'cao cấp': 'Đầy đủ',
    'nội thất': 'Đầy đủ',
}

# Load your CSV file
df = pd.read_csv('data/raws/raw_data.csv')

# Apply the standardization function to the 'legal_document' column
df['legal_document'] = df['legal_document'].apply(lambda x: standardize_column(x, legal_document_mapping))

# Apply the standardization function to the 'interior' column
df['interior'] = df['interior'].apply(lambda x: standardize_column(x, interior_mapping))

# Remove specific substrings and convert to appropriate types
df['num_bedrooms'] = df['num_bedrooms'].str.replace(' phòng', '').astype(float)
df['num_bathrooms'] = df['num_bathrooms'].str.replace(' phòng', '').astype(float)
df['num_floors'] = df['num_floors'].str.replace(' tầng', '').astype(float)
df['entrance'] = df['entrance'].str.replace(' m', '').str.replace(',', '.').astype(float)
df['frontage'] = df['frontage'].str.replace(' m', '').str.replace(',', '.').astype(float)
df['area'] = df['area'].str.replace(' m²', '').str.replace('.', '').str.replace(',', '.').astype(float)

# Standardize 'price' column
def standardize_price(row):
    price = row['price']
    if pd.isna(price):
        return None
    price = price.lower().replace(',', '.').strip()
    if 'tỷ/m²' in price:
        return round(float(price.replace(' tỷ/m²', '')) * 1e9 * row['area'])
    elif 'tỷ' in price:
        return round(float(price.replace(' tỷ', '')) * 1e9)
    elif 'triệu/m²' in price:
        return round(float(price.replace(' triệu/m²', '')) * 1e6 * row['area'])
    elif 'triệu' in price:
        return round(float(price.replace(' triệu', '')) * 1e6)
    else:
        return None

df['price'] = df.apply(standardize_price, axis=1)

# Standardize 'posted_date' column to 'yyyy-mm-dd' format
df['posted_date'] = pd.to_datetime(df['posted_date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

# Add 'price_per_sqm' column (price / area)
df['price_per_sqm'] = round(df['price'] / df['area'])

# # Fill missing numerical values with '0'
# df['num_bedrooms'] = df['num_bedrooms'].fillna(-1)
# df['num_bathrooms'] = df['num_bathrooms'].fillna(-1)
# df['num_floors'] = df['num_floors'].fillna(-1)
# df['entrance'] = df['entrance'].fillna(-1)
# df['frontage'] = df['frontage'].fillna(-1)

# # Standardize data types for cleaned numerical fields
# df['price'] = df['price'].astype(float)
# df['area'] = df['area'].astype(float)
# df['num_bedrooms'] = df['num_bedrooms'].astype(int)
# df['num_bathrooms'] = df['num_bathrooms'].astype(int)
# df['num_floors'] = df['num_floors'].astype(int)
# df['entrance'] = df['entrance'].astype(float)
# df['frontage'] = df['frontage'].astype(float)
# df['price_per_sqm'] = df['price_per_sqm'].astype(float)

# Save the modified dataframe to a new CSV file
df.to_csv('data/results/cleaned_data.csv', index=False)

/var/folders/t4/204x6yq10398qyxz3z4ssvm40000gq/T/ipykernel_89874/1009764359.py:51: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['area'] = df['area'].str.replace(' m²', '').str.replace('.', '').str.replace(',', '.').astype(float)


In [8]:
# Load your CSV file
df = pd.read_csv('data/results/cleaned_data.csv')

# Fill missing numerical values with median values
numerical_columns = ['num_bedrooms', 'num_bathrooms', 'num_floors', 'entrance', 'frontage']

for col in numerical_columns:
    median_value = df[col].median()
    df[col].fillna(median_value, inplace=True)

# Standardize data types for cleaned numerical fields
df['price'] = df['price'].astype(float)
df['area'] = df['area'].astype(float)
df['num_bedrooms'] = df['num_bedrooms'].astype(int)
df['num_bathrooms'] = df['num_bathrooms'].astype(int)
df['num_floors'] = df['num_floors'].astype(int)
df['entrance'] = df['entrance'].astype(float)
df['frontage'] = df['frontage'].astype(float)
df['price_per_sqm'] = df['price_per_sqm'].astype(float)

# Normalize and encode categorical strings
df['type_estate'] = df['type_estate'].str.strip()
df['district'] = df['district'].str.strip()
df['legal_document'] = df['legal_document'].str.strip()
df['interior'] = df['interior'].str.strip()

# Convert categorical fields to numerical codes if needed
label_encoders = {}
categorical_fields = ['type_estate', 'district', 'legal_document', 'interior']

for field in categorical_fields:
    le = LabelEncoder()
    df[field] = le.fit_transform(df[field])
    label_encoders[field] = le  # Store the label encoder if you need to inverse transform later

# Optionally save the cleaned data to a new CSV file
df.to_csv('data/results/standardized_data.csv', index=False)


In [9]:
# Inverse transform example
inverse_data = {
    'pr_id': 39815613,
    'type_estate': 2,
    'district': 4,
    'posted_date': '2024-05-10',
    'area': 65.0,
    'price': 14500000000.0,
    'legal_document': 2,
    'interior': 3,
    'num_bedrooms': -1,
    'num_bathrooms': -1,
    'num_floors': 8,
    'entrance': -1.0,
    'frontage': 5.4,
    'price_per_sqm': 223076923.0
}

# Inverse transform categorical fields
for field, le in label_encoders.items():
    inverse_data[field] = le.inverse_transform([inverse_data[field]])[0]

print(inverse_data)

{'pr_id': 39815613, 'type_estate': 'Nhà riêng', 'district': 'Cầu Giấy', 'posted_date': '2024-05-10', 'area': 65.0, 'price': 14500000000.0, 'legal_document': 'Sổ đỏ/ sổ hồng', 'interior': 'Đầy đủ', 'num_bedrooms': -1, 'num_bathrooms': -1, 'num_floors': 8, 'entrance': -1.0, 'frontage': 5.4, 'price_per_sqm': 223076923.0}
